# Save raw data to 60-20-20 split for train-test-validation sets

In [1]:
import time
import pyspark
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import when, col, to_date, rank, monotonically_increasing_id
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, DateType, FloatType
import pathlib

# spark.stop()

In [2]:
conf = pyspark.SparkConf().setAll([\
    ('spark.app.name', 'ReduceData')])
spark = SparkSession.builder.config(conf=conf)\
    .getOrCreate()

spark.version

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/09 03:11:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


'3.3.1'

### Structs we might need

In [3]:
glucose_data_schema=StructType([StructField('PatientId', StringType(), True),
                                StructField('Value', FloatType(), True),
                                StructField('GlucoseDisplayTime', TimestampType(), True),
                                StructField('GlucoseDisplayTimeRaw', StringType(), True),
                                StructField('GlucoseDisplayDate', DateType(), True),
                                StructField('NumId', IntegerType(), True)
                                ])

raw_schema=StructType([StructField('_c0', IntegerType(),True),
                                StructField('PostDate', TimestampType(),True),
                                StructField('IngestionDate', TimestampType(),True),
                                StructField('PostId', StringType(),True),
                                StructField('PostTime', TimestampType(), True),
                                StructField('PatientId', StringType(), True),
                                StructField('Stream', StringType(), True),
                                StructField('SequenceNumber', StringType(), True),
                                StructField('TransmitterNumber', StringType(), True),
                                StructField('ReceiverNumber', StringType(), True),
                                StructField('RecordedSystemTime', TimestampType(), True),
                                StructField('RecordedDisplayTime', TimestampType(), True),
                                StructField('RecordedDisplayTimeRaw', TimestampType(), True),
                                StructField('TransmitterId', StringType(), True),
                                StructField('TransmitterTime', StringType(), True),
                                StructField('GlucoseSystemTime', TimestampType(), True),
                                StructField('GlucoseDisplayTime', TimestampType(), True),
                                StructField('GlucoseDisplayTimeRaw', StringType(), True),
                                StructField('Value', FloatType(), True),
                                StructField('Status', StringType(), True),
                                StructField('TrendArrow', StringType(), True),
                                StructField('TrendRate', FloatType(), True),
                                StructField('IsBackFilled', StringType(), True),
                                StructField('InternalStatus', StringType(), True),
                                StructField('SessionStartTime', StringType(), True)])

cohortSchema = StructType([StructField('', IntegerType(), True),
                        StructField('UserId', StringType(), True),
                        StructField('Gender', StringType(), True),
                        StructField('DOB', TimestampType(), True),
                        StructField('Age', IntegerType(), True),
                        StructField('DiabetesType', StringType(), True),
                        StructField('Treatment', StringType(), True)
                        ])

### Get all paths in to read from

In [4]:
'''all parquets of the patient data'''
allPaths = [str(x) for x in list(pathlib.Path('/cephfs/data_by_patient').glob('*.parquet')) if 'part-00' in str(x)]

# trainPaths = [str(x) for x in list(pathlib.Path('/cephfs/train_test_val/train_set').glob('*.parquet')) if 'part-00' in str(x)]
# valPaths = [str(x) for x in list(pathlib.Path('/cephfs/train_test_val/val_set').glob('*.parquet')) if 'part-00' in str(x)]
# testPaths = [str(x) for x in list(pathlib.Path('/cephfs/train_test_val/test_set').glob('*.parquet')) if 'part-00' in str(x)]

allPaths.sort()
# trainPaths.sort()
# valPaths.sort()
# testPaths.sort()

# print("train length:", len(trainPaths), "\nvalidation length:", len(valPaths),"\ntest length:", len(testPaths))

allPaths = allPaths[:-1]
len(allPaths)

199

### Read in the Cohort dataframe

In [14]:
# read in cohort dataframe, with Number ID properly labeled
startTime = time.time()

cohortDf = spark.read.options(delimiter=',')\
        .csv('/cephfs/data/cohort.csv', header=True, schema=cohortSchema)\
        .withColumnRenamed('', 'NumId')

print(cohortDf.dtypes)
cohortDf.show(10)
print(time.time() - startTime)

[('NumId', 'int'), ('UserId', 'string'), ('Gender', 'string'), ('DOB', 'timestamp'), ('Age', 'int'), ('DiabetesType', 'string'), ('Treatment', 'string')]


+-----+--------------------+------+-------------------+---+------------+---------+
|NumId|              UserId|Gender|                DOB|Age|DiabetesType|Treatment|
+-----+--------------------+------+-------------------+---+------------+---------+
|    0|5lZPrCk6qk8L6Jw+S...|Female|1931-01-01 00:00:00| 92|    type-two|       no|
|    1|9qY9mZ+GV5Kd/O/NB...|  Male|1937-01-01 00:00:00| 86|    type-two|       no|
|    2|uhsyLhr4Zl6NfGbNB...|Female|1938-01-01 00:00:00| 85|    type-two|       no|
|    3|9uAVHBOgoCJ9hfcrL...|  Male|1938-01-01 00:00:00| 85|    type-two|       no|
|    4|Fyb156jU1edGykL7N...|Female|1939-01-01 00:00:00| 84|    type-two|       no|
|    5|86XfZ0fNI0VWOzWrl...|Female|1939-01-01 00:00:00| 84|    type-two|       no|
|    6|JfJMH1qCpiYNuPOp/...|Female|1940-01-01 00:00:00| 83|    type-two|       no|
|    7|EkW0PD80req7mL/5S...|  Male|1940-01-01 00:00:00| 83|    type-two|       no|
|    8|OyqSKorAj1OPZaevj...|Female|1941-01-01 00:00:00| 82|    type-two|       no|
|   

In [15]:
# make mini dataframe of the string IDs and number IDs
patientIds = cohortDf.select(col('UserId'), col('NumId')).distinct()
patientIds

DataFrame[UserId: string, NumId: int]

### Load in the raw data

In [5]:
# aaaaaaaaaaa??
startTime = time.time()

df = spark.read \
           .schema(glucose_data_schema) \
           .format('parquet') \
           .load(allPaths)

print(time.time() - startTime)

95.27902436256409


In [ ]:
startTime = time.time()

df.printSchema()

print(time.time() - startTime)

root
 |-- PatientId: string (nullable = true)
 |-- Value: float (nullable = true)
 |-- GlucoseDisplayTime: timestamp (nullable = true)
 |-- GlucoseDisplayTimeRaw: string (nullable = true)
 |-- GlucoseDisplayDate: date (nullable = true)
 |-- NumId: integer (nullable = true)

0.021439790725708008


In [7]:
startTime = time.time()

patIds = [i.NumId for i in df.select('NumId').distinct().collect()]
print(len(patIds), "total patients")
print("row count: ", df.count())
df.show()

print(time.time() - startTime)

7957 total patients


row count:  777557880


+--------------------+-----+--------------------+---------------------+------------------+-----+
|           PatientId|Value|  GlucoseDisplayTime|GlucoseDisplayTimeRaw|GlucoseDisplayDate|NumId|
+--------------------+-----+--------------------+---------------------+------------------+-----+
|Uc5BunseQm7FY6Q9G...|  0.0|2022-01-31 19:14:...| 2022-01-31T19:14:...|        2022-01-31| 3945|
|Uc5BunseQm7FY6Q9G...|  0.0|2022-01-31 19:19:...| 2022-01-31T19:19:...|        2022-01-31| 3945|
|Uc5BunseQm7FY6Q9G...|  0.0|2022-01-31 18:59:...| 2022-01-31T18:59:...|        2022-01-31| 3945|
|Uc5BunseQm7FY6Q9G...|  0.0|2022-01-31 19:29:...| 2022-01-31T19:29:...|        2022-01-31| 3945|
|eDp8OyD2YP1ynZNzq...|  0.0|2022-01-31 16:08:...| 2022-01-31T16:08:...|        2022-01-31| 1704|
|eDp8OyD2YP1ynZNzq...|  0.0|2022-02-01 08:33:...| 2022-02-01T08:33:...|        2022-02-01| 1704|
|eDp8OyD2YP1ynZNzq...|  0.0|2022-01-31 22:58:...| 2022-01-31T22:58:...|        2022-01-31| 1704|
|eDp8OyD2YP1ynZNzq...|  0.0|20

### Clean-up

In [6]:
# """the following cell is leslie's cleanup code, yoinked from fill-missing to read-data to here"""
startTime = time.time()

'''get rid of any dates from before the actual start-date of Feb 1, 2022'''
df = df.filter("GlucoseDisplayDate > date'2022-01-31'")

'''replace 0s with NaN and dropna'''
df = df.withColumn("Value", when(col("Value")=="0", None) \
                                                       .otherwise(col("Value")))
df = df.na.drop(subset=['PatientId','Value','GlucoseDisplayTime'])
# df = df.where(df.Value>0)

df = df.withColumn("GlucoseDisplayTime",
                   date_trunc("minute",
                   col("GlucoseDisplayTime")))

'''drop duplicate datetimes for each patient'''
window = Window.partitionBy('GlucoseDisplayTime','PatientId').orderBy('tiebreak')
df = (df
 .withColumn('tiebreak', monotonically_increasing_id())
 .withColumn('rank', rank().over(window))
 .filter(col('rank') == 1).drop('rank','tiebreak')
)

print(time.time() - startTime)

1.283494472503662


### Add in (join) the NumId column

## Split into Train-Test-Val groups of 60-20-20

plan:
* get total count of values per patient
* make 2 cols:
    * int that's the total number of rows that should make up 60% of the patient's data
    * int that's the total number of rows that should make up 80% of the patient's data
* merge that dataframe back onto the main df dataframe
* (is it possible to filter based on another column?)
* add ranks based on patient ID
* filter once that splits at the 60% mark, asking for 'rank'< or > to the 60% mark
    * save the <60% as training
* filter the >60% again on the 80% mark
    * save the <80% as validation
    * save the >80% as test

somewhere in there, get rid of patients with less than 80% data

In [7]:
'''get total counts of values per patient'''
counter = df.groupBy('PatientId').count()
# print("row count: ", counter.count())

'''(?) filter out patients with too little usable data'''
minUsable = 0.80 * (60/5 * 24 * 365)
# temp = counter.filter(col('count') < minUsable)
print("the minimum number of glucose recordings we should be allowing are", minUsable)
# print("but there are only", temp.count(), "patients that have at least that much.")
# counter.write.parquet('temp_data/')

the minimum number of glucose recordings we should be allowing are 84096.0


In [8]:
counter = counter.withColumn("split60",(col("count")* 0.6).cast("Integer"))
counter = counter.withColumn("split80",(col("count")* 0.8).cast("Integer"))

In [9]:
# patientIds = patientIds.join(counter, patientIds.UserId == counter.PatientId)\
#             .select(patientIds.NumId, patientIds.UserId, counter.split60, counter.split80)

counter = counter.withColumnRenamed("PatientId","UserId")

In [10]:
startTime = time.time()

'''get everything into order for ranking/sorting by 60%-20%-20%'''
df = df.join(counter, df.PatientId == counter.UserId)\
            .select(df.NumId, df.PatientId, df.Value, df.GlucoseDisplayTime, \
                    df.GlucoseDisplayTimeRaw, df.GlucoseDisplayDate, \
                    counter.split60, counter.split80)

print(time.time() - startTime)

0.1274428367614746


In [11]:
startTime = time.time()

'''rank, then next cell is filtering'''
window = Window.partitionBy('PatientId').orderBy('tiebreak')
df = df \
 .withColumn('tiebreak', monotonically_increasing_id()) \
 .withColumn('rank', rank().over(window))

print(time.time() - startTime)

0.044648170471191406


In [12]:
'''training set'''
trainSet = df.filter(col('rank') <= col('split60')) \
                 .drop('rank','tiebreak','split60','split80')

'''validation set'''
valSet = df.filter((col('rank') > col('split60')) & (col('rank') <= col('split80'))) \
               .drop('rank','tiebreak','split60','split80')

'''test set'''
testSet = df.filter(col('rank') > col('split80')) \
                .drop('rank','tiebreak','split60','split80')

## Save out into parquet files

In [13]:
'''godspeed'''
startTime = time.time()

# df.repartition('PatientId')\
#     .write.parquet('/cephfs/train_test_val/test/') 
trainSet.repartition('PatientId').write.parquet('/cephfs/train_test_val/train_set/') 
valSet.repartition('PatientId').write.parquet('/cephfs/train_test_val/val_set/') 
testSet.repartition('PatientId').write.parquet('/cephfs/train_test_val/test_set/') 

print(time.time()-startTime)

23/05/09 03:43:30 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/05/09 03:43:32 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/05/09 03:43:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/05/09 03:43:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/05/09 03:43:44 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/05/09 03:43:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/05/09 03:43:50 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/05/09 03:43:52 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/05/09 03:43:56 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/05/09 03:43:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


...

23/05/09 07:18:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/05/09 07:18:05 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/05/09 07:18:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/05/09 07:18:11 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


17300.744101524353


In [14]:
endTime = time.time()
endTime-startTime

17300.751935482025

In [15]:
17300/60/60

4.805555555555555

In [16]:
45887/60/60

12.74638888888889